## Programming part of Homework 3 (Data Structures, Spring 2021)

## Name: 林天佑
## Student ID Number: 108820003

### Programming problem 1
**Un-variate polynomial** of degree $d$ has the form $$c_dx^d+c_{d-1}x^{d-1}+\cdots + c_2x^2+c_1x+c_0,$$ where $c_d\not= 0$. The $c_i$'s are the \emph{coefficients}, and $d, d-1, \cdots$ are the \emph{exponents}. By definition, $d$ is a non-negative integer. In this exercise, we assume that all $c_i$s are integers. We represent each polynomial as a *linear list* of coefficients and would like to have some operations (functions) on the polynomials. The first node in the list represents the first terms in the polynomial, the second node represents the second terms, and so forth.

Each node contains three fields: *the term's coefficient*, *the term's power*, and *a pointer to the next term*. Write a Python program, that first reads the input file, `inFile.txt`, which has three lines and then performs the indicated operation. The first line is an integer representing the operation defined as below. The second line is the first polynomial, and the next line is the second polynomial. The input polynomial, say $4x^3-2x+1$, will be represented as `4x^3-2x+1`. The functions include the following operations:
1. `add`: Add two input polynomials.
2. `subtract`: Subtract the second polynomial from the first one.
3. `multiply`: Multiply two polynomials.
4. `divide`: Divide the first polynomial by the second one and return the quotient.
The input file thus may be

``
2
4x^3-2x+1
3x^2+x+4
``

Your output will be `4x^3-3x^2-3x-3`. Please see the running example in the end of this template.

First, we build up the linked list structure for representing polynomials. Two classes will be defined: `Term` and `Polynomial`.

In [ ]:
from typing import Final, List, Tuple
from string import digits


class Term:
    def __init__(self, coefficient: int or float, exponent: int):
        self.__coefficient = coefficient
        self.__exponent = exponent

    @property
    def coefficient(self) -> int or float:
        return self.__coefficient

    @property
    def exponent(self) -> int:
        return self.__exponent

    @property
    def inverse(self):
        return Term(-self.__coefficient, self.__exponent)

    def __add__(self, other):
        assert self.__exponent == other.exponent
        return Term(self.__coefficient + other.coefficient, self.__exponent)

    def __mul__(self, other):
        return Term(self.__coefficient * other.coefficient, self.__exponent + other.exponent)

    def __cmp__(self, other):
        return self.__coefficient == other.coefficient and self.__exponent == other.exponent

    def __str__(self) -> str:
        if self.__exponent == 0:
            gap_str = ''
            exponent = ''
            coefficient = str(self.__coefficient) if self.__coefficient != 0 else ''
        elif self.__coefficient and self.__exponent == 1:
            gap_str = 'x'
            exponent = ''
            if self.__coefficient == 1:
                coefficient = ''
            elif self.__coefficient == -1:
                coefficient = '-'
            else:
                coefficient = str(self.__coefficient)
        elif self.__coefficient:
            gap_str = 'x^'
            exponent = str(self.__exponent)
            if self.__coefficient == 1:
                coefficient = ''
            elif self.__coefficient == -1:
                coefficient = '-'
            else:
                coefficient = str(self.__coefficient)
        else:
            coefficient = gap_str = exponent = ''

        return coefficient + gap_str + exponent


def str_to_terms(polynomial_str: str) -> Tuple[Term]:
    terms: List[Term] = list()
    negative_sign: Final[str] = '-'
    positive_sign: Final[str] = '+'

    coefficient_str = str()
    coefficient = None

    exponent_str = str()
    exponent = None

    prefix = 1

    length = len(polynomial_str)

    for index, char in enumerate(polynomial_str):
        if char in digits:
            if coefficient is None:
                coefficient_str += char
            else:
                exponent_str += char

            if index == length - 1:
                coefficient = prefix * (int(coefficient_str) if coefficient_str else 0)
                exponent = 0

        elif coefficient_str:
            coefficient = prefix * int(coefficient_str)
            coefficient_str = str()
            prefix = 1

        elif exponent_str:
            exponent = prefix * int(exponent_str)
            exponent_str = str()
            prefix = 1

        elif char == 'x' and coefficient is None and exponent is None:
            coefficient = prefix
            coefficient_str = str()
            prefix = 1

        if char in (negative_sign, positive_sign):
            if char == negative_sign:
                prefix = -1
            if coefficient is not None and exponent is None:
                exponent = 1

        if coefficient is not None and exponent is not None:
            terms.append(Term(coefficient, exponent))
            coefficient = None
            exponent = None

    return tuple(terms)


class Polynomial:
    def __init__(self, terms: Tuple[Term] or str):
        if isinstance(terms, str):
            self.__terms = str_to_terms(terms)
        else:
            self.__terms = terms
        self.__simplification()

    def __str__(self) -> str:
        return ''.join(
            (str(term) if index == 0 else (('+' if term.coefficient > 0 else '') + str(term)))
            for index, term in enumerate(self.__terms)
        )

    __repr__ = __str__

    def __simplification(self):
        term_dict = dict()

        for term in self.__terms:
            if term.exponent not in term_dict:
                term_dict[term.exponent] = list()
            term_dict[term.exponent].append(term)

        for exponent, terms in list(term_dict.items()):
            merged_term = terms[0]
            for term in terms[1:]:
                merged_term += term
            if merged_term.coefficient == 0:
                del term_dict[exponent]
            else:
                term_dict[exponent] = merged_term

        self.__terms = sorted(term_dict.values(), key=lambda _term: _term.exponent, reverse=True)

    @property
    def raw_terms(self) -> Tuple[Term]:
        return self.__terms

    @property
    def leading_exponent(self) -> int:
        return self.__terms[0].exponent

    def __add__(self, other):
        return Polynomial(tuple([ours for ours in self.__terms] + [his for his in other.raw_terms]))

    def __sub__(self, other):
        return Polynomial(tuple([ours for ours in self.__terms] + [his.inverse for his in other.raw_terms]))

    def __mul__(self, other):
        results: List[Term] = list()
        for my_term in self.__terms:
            for his_term in other.raw_terms:
                results.append(my_term * his_term)

        return Polynomial(tuple(results))

    def __truediv__(self, other):
        quotient: List[Term] = list()
        remainder = self.raw_terms

        while remainder[0].exponent >= other.raw_terms[0].exponent:
            new_quotient_term = Term(
                remainder[0].coefficient / other.raw_terms[0].coefficient,
                remainder[0].exponent - other.raw_terms[0].exponent
            )

            quotient.append(new_quotient_term)
            remainder = (Polynomial(tuple(remainder)) - Polynomial(tuple([new_quotient_term])) * other).raw_terms

        return Polynomial(tuple(quotient)), Polynomial(tuple(remainder))


Then, we may provide the functions for helping read and parse the input file to have the input operation and polynomials.
The `read_lines()` function reads the lines into and returns a list of strings. 
Function `read_string(s)` parses an input string to a polynomial with linked list representation. 

In [ ]:
# functions for reading and parsing the input file to have the input polynomials and operation
# function for reading lines in the input text file into a list of strings      
def read_lines(file='inFile.txt') -> Tuple[str]:
    result: List[str] = list()
    with open(file, 'r') as f:
        for line in f:
            result.append(line.replace('\n', ''))
    return tuple(result)


Below, the functions for polynomial operations with two input polynomials are provided:
1. `add()`: Add two input polynomials.
2. `subtract()`: Subtract the second polynomial from the first one.
3. `multiply()`: Multiply two polynomials.
4. `divide()`: Divide the first polynomial by the second one and return the quotient and remainder.

**Note that** since `divide()` returns two resulting polynomials. We therefore have all the functions for operations return two polynomials. If there is only one resulting polynomial, we use `None` object for the second polynomail to return.

In [ ]:
# functions for polynomial operations
# adding two polynomials
def add(poly1: Polynomial, poly2: Polynomial) -> Polynomial:
    return poly1 + poly2


# subtracting poly2 from poly1
def subtract(poly1: Polynomial, poly2: Polynomial) -> Polynomial:
    return poly1 - poly2


# multiplying two polynomials
def multiply(poly1: Polynomial, poly2: Polynomial) -> Polynomial:
    return poly1 * poly2


# dividing poly1 by poly2 and then returning the quotient and remainder
def divide(poly1: Polynomial, poly2: Polynomial) -> Tuple[Polynomial]:
    return poly1 / poly2


Last, we perform the operations according to the input file, `inFile.txt`. Function `poly_operation()` can be as the main program entry and first derive the operation with the derived list of strings from function `read_lines()`. Then, `operation_selection()` is called to perform the corresponding operation. **Note that** there will be two polynomials returned for each operation. Last, it prints out the result. The whole program will be executed by calling `poly_operation()` with the input file, `inFile.txt`. One can change the content in the input file for different cases. 

***Basically, the following cell can be kept without change if you would like to follow it for programming. Of course, you can have your own code for this part. However, the function name of the main program entry, `poly_operation()` can not be changed.***

In [ ]:
# main program area
# function to call the corresponding operation and return two polynomial
def operation_selection(operation: int, poly1: Polynomial, poly2: Polynomial) -> Polynomial:
    switcher = {
        1: add,
        2: subtract,
        3: multiply,
        4: divide,
    }
    # Get the function from switcher dictionary
    func = switcher.get(operation, lambda: "nothing")

    # Execute the function
    return func(poly1, poly2)


# program entry
# function for starting the task
def poly_operation() -> None:
    #
    # read the input information from the default input text file
    #
    strings = read_lines()

    #
    # obtain the operation: 1. add; 2. subtract; 3. Multiply; 4. Divide
    # and print it out
    #
    operation = int(strings[0])
    operations = {
        1: 'add',
        2: 'subtract',
        3: 'multiply',
        4: 'divide'
    }
    print(strings[1], operations.get(operation), strings[2])

    #
    # parse strings 1 and 2 to derive the input polynomials and represent them with
    # linked lists
    #
    poly1 = Polynomial(strings[1])
    poly2 = Polynomial(strings[2])

    #
    # perform the operation and two polynomials are returned.
    #
    r1 = operation_selection(operation, poly1, poly2)
    if isinstance(r1, tuple):
        r2 = r1[1]
        r1 = r1[0]
    else:
        r2 = None

    #
    # print out the result
    #
    if operation == 4:
        print("The quotient is:", r1)
        print("The remainder is:", r2)
    else:
        print("The result is:", r1)

    # execute the program with the input file inFile.txt


if __name__ == '__main__':
    poly_operation()